# Using submodels in PyBaMM
In this notebook we show how to modify existing models by swapping out submodels, and how to build your own model from scratch using exisitng submodels. To see all of the models and submodels available in PyBaMM, please take a look at the documentation [here](https://pybamm.readthedocs.io/en/latest/source/models/index.html).

## Changing a submodel in an exisiting battery model
PyBaMM is designed to be a flexible modelling package that allows users to easily compare different models and numerical techniques within a common framework. Battery models within PyBaMM are built up using a number of submodels that describe different physics included within the model, such as mass conservation in the electrolyte or charge conservation in the solid. For ease of use, a number of popular battery models are pre-configured in PyBaMM. As an example, we look at the Single Particle Model (for more information see [here](./models/SPM.ipynb)). First we import PyBaMM and any other packages we need

In [1]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

When you load a model in PyBaMM it builds by default. Building the model sets all of the model variables and sets up any variables which are coupled between different submodels: this is the process which couples the submodels together and allows one submodel to access variables from another. If you would like to swap out a submodel in an exisitng battery model you need to load it without building it by passing the keyword `build=False`

In [2]:
model = pybamm.lithium_ion.SPMe(build=False)

This collects all of the submodels which make up the SPM, but doesn't build the model. Now you are free to swap out one submodel for another. For instance, you may want to assume that diffusion within the negative particles is infinitely fast, so that the PDE describing diffusion is replaced with an ODE for the uniform particle concentration. To change a submodel you simply update the dictionary entry

In [3]:
model.submodels["negative particle"] = pybamm.particle.FickianSingleParticle(model.param, "Negative")
model.submodels["positive particle"] = pybamm.particle.FickianSingleParticle(model.param, "Positive")
model.submodels["particle cracking"] = pybamm.particle_cracking.CrackPropagation(model.param, "Negative")

Building the model also sets up the equations, boundary and initial conditions for the model.

In [4]:
model.build_model()

We can get the default parameters for the model and update them with the parameters required by the cracking model. Eventually, we would like these to be added to their won chemistry (you might need to adjust the path to the parameters file to your system).

In [5]:
param = model.default_parameter_values

import pandas as pd
mechanics = pd.read_csv("pybamm/input/parameters/lithium-ion/mechanicals/lico2_graphite_Ai2020/parameters.csv", 
                        index_col=0, comment="#", skip_blank_lines=True, header=None)[1][1:].dropna().astype(float).to_dict()
param.update(mechanics, check_already_exists=False)

Now the model can be processed and solved in the usual way, and we still have access to model defaults such as the default geometry and default spatial methods

In [6]:
# create geometry
geometry = model.default_geometry

# load parameter values and process model and geometry
param.process_model(model)
param.process_geometry(geometry)

# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# solve model
t_eval = np.linspace(0, 3600, 100)
solution = model.default_solver.solve(model, t_eval)

# plot
quick_plot = pybamm.QuickPlot(solution)
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

Plot the results.

In [7]:
# extract voltage
stress_t_n_surf = solution['Negative particle surface tangential stress']
c_s_n = solution['Negative particle concentration']
t_all = solution["Time [s]"].entries
x = solution["x [m]"].entries[:, 0]

# plot
c_s_n = solution['Negative particle concentration']
r_n = solution["r_n [m]"].entries[:, 0, 0]

def plot_concentrations(t):
    f, (ax1, ax2) = plt.subplots(1, 2 ,figsize=(10,5))
    ax1.plot(t_all, stress_t_n_surf(t=t_all,x=x[0]))
    ax1.set_xlabel(r'$t$ [s]')
    ax1.set_ylabel('$\sigma_t/E_n$')
    ax1.set_ylim(0, 0.0015)
    
    plot_c_n, = ax2.plot(r_n, c_s_n(r=r_n,t=t,x=x[0]))  # can evaluate at arbitrary x (single representative particle)
    ax2.set_ylabel('Negative particle concentration')
    ax2.set_xlabel(r'$r_n$ [m]')
    ax2.set_ylim(0, 1)
    plt.show()
    
import ipywidgets as widgets
widgets.interact(plot_concentrations, t=widgets.FloatSlider(min=0,max=3600,step=10,value=0));

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=10.0), Output()), _dom_classes=…

In [8]:
label = ["SPM"]
output_variables = ["Negative particle surface tangential stress", "Negative particle concentration"]
quick_plot = pybamm.QuickPlot(solution, output_variables, label)
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…